In [1]:
from TransitGraphStatic import TransitGraph, EnhTransitGraph

tram=TransitGraph(trips=r"D:\osm2gtfs\ru_mow_osmgrabber\tram_trips.json", stops=r"D:\osm2gtfs\ru_mow_osmgrabber\tram_stops.json", s2s=r"D:\osm2gtfs\ru_mow_osmgrabber\tram_s2s.json", speed=21, type='tram')
print(len(tram.graph.nodes()), len(tram.graph.edges()))
subway=TransitGraph(trips=r"D:\osm2gtfs\ru_mow_osmgrabber\subway_trips.json", stops=r"D:\osm2gtfs\ru_mow_osmgrabber\subway_stops.json", s2s=r"D:\osm2gtfs\ru_mow_osmgrabber\subway_s2s.json", speed=30, type='subway')
print(len(subway.graph.nodes()), len(subway.graph.edges()))
commuter=TransitGraph(trips=r"D:\osm2gtfs\ru_mow_osmgrabber\commuter_trips.json", stops=r"D:\osm2gtfs\ru_mow_osmgrabber\commuter_stops.json", s2s=r"D:\osm2gtfs\ru_mow_osmgrabber\commuter_s2s.json", speed=35, type='commuter')
print(len(commuter.graph.nodes()), len(commuter.graph.edges()))

831 1981
292 518
264 326


In [2]:
from TransitGraphStatic import PedestrianGraph
pedestrian=PedestrianGraph(r'D:\capitals_accessibility\moscow-latest.osm.pbf').graph

In [3]:
print(len(pedestrian.nodes()), len(pedestrian.edges()))

6748815 997692


In [4]:
pedestrian.nodes()[0]

(55.9951147, 37.2075319)

In [5]:
from TransitGraphStatic import EnhTransitGraph
enh=EnhTransitGraph([commuter.graph, tram.graph, subway.graph], pedestrian=pedestrian)
print(len(enh.graph.nodes()), len(enh.graph.edges()))

2
6750202 1003291


In [ ]:
cycling_edges=[enh.graph.get_edge_data(*e) for e in enh.graph.edge_list() if 'type' not in enh.graph.get_edge_data(*e)]
cycling_edges

In [ ]:
print(pedestrian.nodes()[0])

In [ ]:
cycling_nodes=[enh.graph.get_node_data(n) for n in enh.graph.node_indexes() if 'type' not in enh.graph.get_node_data(n)]
cycling_nodes

In [ ]:
print(tram.graph.edge_list())

In [ ]:
import geopandas as gpd
import pandas as pd
import shapely
route=[]

stations=[]
for node in subway.graph.node_indices():
    data=subway.graph[node]
    data['node_idx']=node
    stations.append(data)
for node in tram.graph.node_indices():
    data=tram.graph[node]
    data['node_idx']=node
    stations.append(data)
for node in commuter.graph.node_indices():
    data=commuter.graph[node]
    data['node_idx']=node
    stations.append(data)
stations_gdf=gpd.GeoDataFrame(stations)
stations_gdf['geom']=stations_gdf['geom'].apply(lambda x: shapely.from_wkt(x))
stations_gdf.set_geometry('geom', crs='EPSG:4326', inplace=True)

for edge in subway.graph.edge_list():
    #if subway.graph.get_edge_data(*edge)['ref']=='1':
    route.append(subway.graph.get_edge_data(*edge))
for edge in tram.graph.edge_list():
    #if subway.graph.get_edge_data(*edge)['ref']=='1':
    route.append(tram.graph.get_edge_data(*edge))
for edge in commuter.graph.edge_list():
    #if subway.graph.get_edge_data(*edge)['ref']=='1':
    route.append(commuter.graph.get_edge_data(*edge))
route_gdf=gpd.GeoDataFrame(route)
route_gdf['geom']=route_gdf['geom'].apply(lambda x: shapely.from_wkt(x))
route_gdf.set_geometry('geom', crs='EPSG:4326', inplace=True)

route_gdf.explore('traveltime')

In [ ]:
d4_edges=[commuter.graph.get_edge_data(*d4_e) for d4_e in commuter.graph.edge_list() if commuter.graph.get_edge_data(*d4_e)['ref']=='D4']
d4_edges

In [ ]:
d4_edges=[enh.graph.get_edge_data(*d4_e) for d4_e in enh.graph.edge_list() if enh.graph.get_edge_data(*d4_e)['ref']=='D4']
d4_edges

In [ ]:
print(len(subway.graph.nodes()), len(subway.graph.edges()))
print(len(commuter.graph.nodes()), len(commuter.graph.edges()))
print(len(tram.graph.nodes()), len(tram.graph.edges()))

In [ ]:
print(len(enh.graph.nodes()), len(enh.graph.edges()))

In [ ]:
import geopandas as gpd
import pandas as pd
import shapely
route=[]

stations=[]
for node in enh.graph.node_indices():
    data=enh.graph[node]
    data['node_idx']=node
    stations.append(data)

stations_gdf=gpd.GeoDataFrame(stations)
stations_gdf['geom']=stations_gdf['geom'].apply(lambda x: shapely.from_wkt(x))
stations_gdf.set_geometry('geom', crs='EPSG:4326', inplace=True)

for edge in enh.graph.edge_list():
    route.append(enh.graph.get_edge_data(*edge))

route_gdf=gpd.GeoDataFrame(route)
route_gdf['geom']=route_gdf['geom'].apply(lambda x: shapely.from_wkt(x))
route_gdf.set_geometry('geom', crs='EPSG:4326', inplace=True)

route_gdf.plot('type')

In [ ]:
stations_gdf.explore()

In [ ]:
import rustworkx as rx
start = 215
end = 238
path=rx.dijkstra_shortest_paths(enh.graph, start, end)
#print(next(path.keys()))
print(path)
path_edges=[]
total_time=0
for node_i in range(len(path[end])-1):
    node_now=path[end][node_i]
    node_next=path[end][node_i+1]
    edge=enh.graph.get_edge_data(node_now, node_next)
    print(enh.graph[node_now]['name'], ' - ', enh.graph[node_next]['name'])
    print(edge['traveltime'])
    total_time+=edge['traveltime']
    #edge_attr=subway_graph.edges()[edge]
    #total_time+=edge_attr
    #edge_geom=next((s2s_e for s2s_e in s2s if s2s_e['edge_idx'] == edge), None)
    #node_inf=next((stop for stop in stops if stop['node_idx'] == node), None)
    
    #print(node_inf)
    #print(edge_attr)
    path_edges.append(shapely.from_wkt(edge['geom']))
print('Total travel time:', total_time)

In [ ]:
path_edges

In [ ]:
route_gdf

In [ ]:
print(enh.graph.edge_list())

In [ ]:
print(enh.graph.node_indexes())

In [ ]:
def raptor(graph, source, target, max_rounds):
    """
    Ищет маршруты от source до target с использованием RAPTOR.
    
    :param source: Начальная остановка
    :param target: Конечная остановка
    :param max_rounds: Максимальное число раундов для поиска
    :return: Кратчайшие пути с минимальным количеством пересадок
    """
    # Инициализация
    earliest_arrival = {stop: float('inf') for stop in graph.node_indexes()}
    earliest_arrival[source] = 0

    marked_stops = set()
    marked_stops.add(source)

    journeys = []  # Список найденных маршрутов
    paths = {stop: [] for stop in graph.node_indexes()}  # Хранит путь до каждой остановки
    paths[source] = [(None, source)]  # Начинаем с исходной остановки

    for round_number in range(max_rounds):
        # Для каждого раунда инициализируем набор измененных остановок
        updated_stops = set()
        
        for stop in marked_stops:
            # Для каждой остановки проверяем все маршруты, которые через неё проходят
            for neighbor in graph.neighbors(stop):
                edge_data = graph.get_edge_data(stop, neighbor)
                edge_type = edge_data['type']

                if edge_type in ('interchange', 'ginterchange'):
                    route = f'ic_{round_number}'
                else:
                    route = edge_data['trip_id']

                if earliest_arrival[neighbor] > earliest_arrival[stop] + 1:
                    earliest_arrival[neighbor] = earliest_arrival[stop] + 1
                    updated_stops.add(neighbor)

                    # Обновляем путь до остановки
                    paths[neighbor] = paths[stop] + [(route, neighbor)]

                    if neighbor == target:
                        # Сохраняем найденный маршрут
                        journeys.append(paths[neighbor])

        # Обновляем список остановок для следующего раунда
        marked_stops = updated_stops
    
    print(marked_stops)
    return journeys

journey = raptor(enh.graph, 544, 543, 8)
print("Найденные маршруты:")
for path in journey:
    for node in path:
        print(enh.graph[node[1]])


In [ ]:
import rustworkx as rx
start = 100
end = 1
path=rx.dijkstra_shortest_paths(subway.graph, start, end)
#print(next(path.keys()))
print(path)
path_edges=[]
total_time=0
for node_i in range(len(path[end])-1):
    node_now=path[end][node_i]
    node_next=path[end][node_i+1]
    edge=subway.graph.get_edge_data(node_now, node_next)
    print(subway.graph[node_now]['name'], ' - ', subway.graph[node_next]['name'])
    print(edge['traveltime'])
    total_time+=edge['traveltime']
    #edge_attr=subway_graph.edges()[edge]
    #total_time+=edge_attr
    #edge_geom=next((s2s_e for s2s_e in s2s if s2s_e['edge_idx'] == edge), None)
    #node_inf=next((stop for stop in stops if stop['node_idx'] == node), None)
    
    #print(node_inf)
    #print(edge_attr)
    #path_edges.append()
print(total_time)

In [ ]:
print(subway.graph[200])